<a href="https://colab.research.google.com/github/Adla-Zubair/Adla-Zubair/blob/main/Resampling_using_mne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

In [ ]:
!pip install pyedflib

In [16]:
import mne
from mne.io import RawArray
#from mne.channels import Info
from pyedflib import highlevel

In [36]:
file = '/content/FA7310SO_0001.edf'

In [ ]:
raw = mne.io.read_raw_edf(file, preload= True)

In [38]:
raw_eeg = raw.resample(sfreq=250)

In [ ]:
# Save the modified EEG data to your desktop
output_path = '/content/raw_eeg.fif'  # Replace with your desired path
raw.save(output_path, overwrite=True)

In [ ]:
output = mne.export('raw_resampled', raw, fmt='edf', physical_range='auto', add_ch_type=True, *, overwrite=False, verbose=None)

In [39]:
import pyedflib # pip install pyedflib
from pyedflib import highlevel # new high-level interface
from pyedflib import FILETYPE_BDF, FILETYPE_BDFPLUS, FILETYPE_EDF, FILETYPE_EDFPLUS
from datetime import datetime, timezone, timedelta
import mne
import os

def _stamp_to_dt(utc_stamp):
    """Convert timestamp to datetime object in Windows-friendly way."""
    if 'datetime' in str(type(utc_stamp)): return utc_stamp
    # The min on windows is 86400
    stamp = [int(s) for s in utc_stamp]
    if len(stamp) == 1:  # In case there is no microseconds information
        stamp.append(0)
    return (datetime.fromtimestamp(0, tz=timezone.utc) +
            timedelta(0, stamp[0], stamp[1]))  # day, sec, μs


def write_mne_edf(raw_eeg, fname = 'raw_resampled', picks=None, tmin=None, tmax=None,
                  overwrite=False):
    """
    Saves the raw content of an MNE.io.Raw and its subclasses to
    a file using the EDF+/BDF filetype
    pyEDFlib is used to save the raw contents of the RawArray to disk
    Parameters
    ----------
    mne_raw : mne.io.Raw
        An object with super class mne.io.Raw that contains the data
        to save
    fname : string
        File name of the new dataset. This has to be a new filename
        unless data have been preloaded. Filenames should end with .edf
    picks : array-like of int | None
        Indices of channels to include. If None all channels are kept.
    tmin : float | None
        Time in seconds of first sample to save. If None first sample
        is used.
    tmax : float | None
        Time in seconds of last sample to save. If None last sample
        is used.
    overwrite : bool
        If True, the destination file (if it exists) will be overwritten.
        If False (default), an error will be raised if the file exists.
    """
    if not issubclass(type(raw_eeg), mne.io.BaseRaw):
        raise TypeError('Must be mne.io.Raw type')
    if not overwrite and os.path.exists(fname):
        raise OSError('File already exists. No overwrite.')

    # static settings
    has_annotations = True if len(raw_eeg.annotations)>0 else False
    if os.path.splitext(fname)[-1] == '.edf':
        file_type = FILETYPE_EDFPLUS if has_annotations else FILETYPE_EDF
        dmin, dmax = -32768, 32767
    else:
        file_type = FILETYPE_BDFPLUS if has_annotations else FILETYPE_BDF
        dmin, dmax = -8388608, 8388607

    print('saving to {}, filetype {}'.format(fname, file_type))
    sfreq = raw_eeg.info['sfreq']
    date = _stamp_to_dt(raw_eeg.info['meas_date'])

    if tmin:
        date += timedelta(seconds=tmin)
    # no conversion necessary, as pyedflib can handle datetime.
    #date = date.strftime('%d %b %Y %H:%M:%S')
    first_sample = int(sfreq*tmin)
    last_sample  = int(sfreq*tmax) if tmax is not None else None


    # convert data
    channels = raw_eeg.get_data(picks,
                                start = first_sample,
                                stop  = last_sample)

    # convert to microvolts to scale up precision
    channels *= 1e6

    # set conversion parameters
    n_channels = len(channels)

    # create channel from this
    try:
        f = pyedflib.EdfWriter(fname,
                               n_channels=n_channels,
                               file_type=file_type)

        channel_info = []

        ch_idx = range(n_channels) if picks is None else picks
        keys = list(raw_eeg._orig_units.keys())
        for i in ch_idx:
            try:
                ch_dict = {'label': raw_eeg.ch_names[i],
                           'dimension': raw_eeg._orig_units[keys[i]],
                           'sample_rate': raw_eeg._raw_extras[0]['n_samps'][i],
                           'physical_min': raw_eeg._raw_extras[0]['physical_min'][i],
                           'physical_max': raw_eeg._raw_extras[0]['physical_max'][i],
                           'digital_min':  raw_eeg._raw_extras[0]['digital_min'][i],
                           'digital_max':  raw_eeg._raw_extras[0]['digital_max'][i],
                           'transducer': '',
                           'prefilter': ''}
            except:
                ch_dict = {'label': raw_eeg.ch_names[i],
                           'dimension': raw_eeg._orig_units[keys[i]],
                           'sample_rate': sfreq,
                           'physical_min': channels.min(),
                           'physical_max': channels.max(),
                           'digital_min':  dmin,
                           'digital_max':  dmax,
                           'transducer': '',
                           'prefilter': ''}

            channel_info.append(ch_dict)
        f.setPatientCode(raw_eeg._raw_extras[0]['subject_info'].get('id', '0'))
        f.setPatientName(raw_eeg._raw_extras[0]['subject_info'].get('name', 'noname'))
        f.setTechnician('mne-gist-save-edf-skjerns')
        f.setSignalHeaders(channel_info)
        f.setStartdatetime(date)
        f.writeSamples(channels)
        for annotation in raw_eeg.annotations:
            onset = annotation['onset']
            duration = annotation['duration']
            description = annotation['description']
            f.writeAnnotation(onset, duration, description)

    except Exception as e:
        raise e
    finally:
        f.close()
    return True

In [41]:
output_path = '/content/raw_eeg.edf'

In [26]:
import pyedflib

# Convert MNE Raw data to pyedflib RawEDF object
edf_data = raw.get_data()
edf_channels = raw.info['ch_names']
sample_frequency = raw.info['sfreq']
n_channels, n_samples = edf_data.shape


In [ ]:

# Create an EDF file
edf_file = pyedflib.EdfWriter('raw_eeg.edf', n_channels=n_channels, file_type=pyedflib.FILETYPE_EDFPLUS)

# Add channel information
# Create EdfSignalInfo objects for each channel and set labels
channel_info = [pyedflib.EdfSignalInfo(label=edf_channels, sample_rate=sample_frequency, physical_max=edf_data[i].max(),
                                       physical_min=edf_data[i].min(), digital_max=32767, digital_min=-32768,
                                       transducer='')]
for i in range(n_channels):
    edf_file.setSignalHeader(i, channel_info[i])

# Write EEG data to the file
edf_file.writeSamples(edf_data)

# Close the EDF file
edf_file.close()






In [42]:
#Close the EDF file
edf_file.close()